In [1]:
%load_ext autoreload
%autoreload 2

from torch_snippets import *

In [2]:
x = AttrDict({'abc': {'dev': 10}})

In [3]:
x.abc.dev

'dev' in x.abc

True

In [4]:
inspect(x)

	Dict Of 1 items
		ABC:
		Dict Of 1 items
			DEV:
			int: 10


In [6]:
x.drop('dev')
x.pretty()

{
    "abc": {}
}


In [10]:
pdf = PDF('/Users/yeshwanth.y/Documents/personal-vault/Career/Documents/618_YeshwanthReddyYerraguntla _CEE.pdf')
len(pdf)

2